# **Face Recognition System with Personal Information Retrieval Using Deep Learning**

**Introduction:**

This project implements a robust face recognition system leveraging deep learning techniques and the FaceNet architecture for feature extraction. The system identifies individuals from images by extracting facial embeddings and classifying them using a Multi-Layer Perceptron (MLP) neural network. Beyond simple recognition, the system can also retrieve and display relevant personal information such as age, gender, nationality, role, and other attributes stored in a structured CSV file. The pipeline supports both single-image inference and batch processing of multiple images, enabling flexible real-world applications such as access control, actor identification, or personalized recommendations. The solution is designed to be efficient, scalable, and compatible with GPU acceleration for faster training and inference.

**Imports and Library Setup for Face Recognition and Classification Pipeline**

In [ ]:
import os
import glob
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

import face_recognition
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


**Face Recognition Pipeline: Data Preparation, Model Training, Evaluation, and Inference**

In [ ]:


DATASET_DIR = r"C:/Users/nandhudivya/Desktop/project/dataset"
SAVEDIR = "C:/Users/nandhudivya/Desktop/project/saved_models"
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-3
TEST_SIZE = 0.2
RANDOM_STATE = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(SAVEDIR, exist_ok=True)


# Extractraction of face encodings for each image

def extract_encodings(dataset_dir):
    X = []
    y = []

    persons = sorted([d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))])
    if not persons:
        raise RuntimeError(f"No person subfolders found in {dataset_dir}. Expect dataset/person_name/*.jpg")

    for person in persons:
        person_dir = os.path.join(dataset_dir, person)
        image_files = []

        for ext in ("*.jpg", "*.jpeg", "*.png"):
            image_files.extend(glob.glob(os.path.join(person_dir, ext)))
        if not image_files:
            print(f"Warning: no images found in {person_dir}")
            continue

        for img_path in image_files:

            try:
                img = face_recognition.load_image_file(img_path)
            except Exception as e:
                print(f"Could not load {img_path}: {e}")
                continue


            encs = face_recognition.face_encodings(img)
            if len(encs) == 0:

                print(f"⚠️  No face detected in {img_path}; skipping.")
                continue


            encoding = encs[0]
            X.append(encoding)
            y.append(person)

    X = np.array(X)
    y = np.array(y)
    print(f"Extracted {len(X)} encodings for {len(np.unique(y))} people.")
    return X, y


# Building dataset and encoding labels

def build_dataset(X, y, test_size=TEST_SIZE):
    le = LabelEncoder()
    y_enc = le.fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=test_size, random_state=RANDOM_STATE, stratify=y_enc)

    X_train_t = torch.tensor(X_train, dtype=torch.float32)
    y_train_t = torch.tensor(y_train, dtype=torch.long)
    X_test_t = torch.tensor(X_test, dtype=torch.float32)
    y_test_t = torch.tensor(y_test, dtype=torch.long)
    return (X_train_t, y_train_t), (X_test_t, y_test_t), le


#  Simple MLP classifier (ANN):
class FaceNetMLP(nn.Module):
    def __init__(self, input_dim=128, hidden_dims=(256,128), num_classes=2, dropout=0.3):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev, h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev = h
        layers.append(nn.Linear(prev, num_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


# Training loop

def train_model(model, train_loader, val_loader, epochs=EPOCHS, lr=LR, device=DEVICE):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_val_acc = 0.0
    best_state = None

    for epoch in range(1, epochs+1):
        model.train()
        running_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * xb.size(0)

        avg_loss = running_loss / len(train_loader.dataset)

        # Model-Validation
        model.eval()
        preds = []
        trues = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                pred = torch.argmax(logits, dim=1)
                preds.extend(pred.cpu().numpy())
                trues.extend(yb.cpu().numpy())
        val_acc = accuracy_score(trues, preds)
        print(f"Epoch {epoch}/{epochs} — loss: {avg_loss:.4f}, val_acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = model.state_dict()

    # best-Load
    if best_state:
        model.load_state_dict(best_state)
    print(f"Best val acc: {best_val_acc:.4f}")
    return model


#Evaluation helper

def evaluate(model, X_test_t, y_test_t, le, device=DEVICE):
    model.eval()
    model.to(device)
    with torch.no_grad():
        xb = X_test_t.to(device)
        logits = model(xb)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        trues = y_test_t.numpy()

    print("Accuracy:", accuracy_score(trues, preds))
    print("Classification report:")
    print(classification_report(trues, preds, target_names=le.classes_))
    print("Confusion matrix:")
    print(confusion_matrix(trues, preds))


# single image-Inference

def predict_image(model, image_path, le, device=DEVICE):
    img = face_recognition.load_image_file(image_path)
    encs = face_recognition.face_encodings(img)
    if len(encs) == 0:
        print("No face detected.")
        return None
    enc = torch.tensor(encs[0], dtype=torch.float32).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        logits = model(enc)
        prob = torch.softmax(logits, dim=1)
        pred_idx = int(torch.argmax(prob, dim=1).cpu().numpy()[0])
        confidence = float(prob[0, pred_idx].cpu().numpy())
    name = le.inverse_transform([pred_idx])[0]
    return name, confidence

if __name__ == "__main__":
    print("Device:", DEVICE)
    X, y = extract_encodings(DATASET_DIR)
    if len(X) == 0:
        raise RuntimeError("No encodings extracted. Check dataset and face detection.")

    (X_train_t, y_train_t), (X_test_t, y_test_t), le = build_dataset(X, y)

    num_classes = len(le.classes_)
    print("Classes:", le.classes_)

    train_ds = TensorDataset(X_train_t, y_train_t)
    test_ds = TensorDataset(X_test_t, y_test_t)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    # Creation of Model
    model = FaceNetMLP(input_dim=128, hidden_dims=(256,128), num_classes=num_classes, dropout=0.3)

    # Model-training:
    model = train_model(model, train_loader, test_loader, epochs=EPOCHS, lr=LR, device=DEVICE)

    # Model-evaluation:
    evaluate(model, X_test_t, y_test_t, le, device=DEVICE)

    # Model-Save & Label encoding:
    torch.save(model.state_dict(), os.path.join(SAVEDIR, "ann_face_recognition.pt"))
    with open(os.path.join(SAVEDIR, "label_encoder.pkl"), "wb") as f:
        pickle.dump(le, f)
    print("Saved model and label encoder to", SAVEDIR)


    sample_image = os.path.join(DATASET_DIR, os.listdir(DATASET_DIR)[0])



Device: cuda
⚠️  No face detected in C:/Users/nandhudivya/Desktop/project/dataset\Angelina Jolie\044_512dfd33.jpg; skipping.
⚠️  No face detected in C:/Users/nandhudivya/Desktop/project/dataset\Angelina Jolie\049_4d6df392.jpg; skipping.
⚠️  No face detected in C:/Users/nandhudivya/Desktop/project/dataset\Angelina Jolie\092_26130bb1.jpg; skipping.
⚠️  No face detected in C:/Users/nandhudivya/Desktop/project/dataset\Angelina Jolie\095_0be163a1.jpg; skipping.
⚠️  No face detected in C:/Users/nandhudivya/Desktop/project/dataset\Brad Pitt\041_cc0957bf.jpg; skipping.
⚠️  No face detected in C:/Users/nandhudivya/Desktop/project/dataset\Brad Pitt\048_185402c6.jpg; skipping.
⚠️  No face detected in C:/Users/nandhudivya/Desktop/project/dataset\Brad Pitt\060_136e5ef5.jpg; skipping.
⚠️  No face detected in C:/Users/nandhudivya/Desktop/project/dataset\Brad Pitt\076_75b9dd73.jpg; skipping.
⚠️  No face detected in C:/Users/nandhudivya/Desktop/project/dataset\Brad Pitt\095_1104d364.jpg; skipping.
⚠️  

**Single Image Face Recognition: Model Inference and Prediction**

In [ ]:
#Config
MODEL_PATH = "C:/Users/nandhudivya/Desktop/project/saved_models/ann_face_recognition.pt"
ENCODER_PATH = "C:/Users/nandhudivya/Desktop/project/saved_models/label_encoder.pkl"
TEST_IMAGE = "C:/Users/nandhudivya/Desktop/project/test_img/anjel.jpg"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Model Loading and label encoding

with open(ENCODER_PATH, "rb") as f:
    label_encoder = pickle.load(f)

num_classes = len(label_encoder.classes_)
model = FaceNetMLP(input_dim=128, num_classes=num_classes)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

print(f"✅ Model and label encoder loaded successfully. Classes: {label_encoder.classes_}")

# Extraction

try:
    image = face_recognition.load_image_file(TEST_IMAGE)
except Exception as e:
    raise SystemExit(f"❌ Could not load test image: {e}")

encodings = face_recognition.face_encodings(image)
if len(encodings) == 0:
    raise SystemExit("⚠️ No face found in the test image.")

encoding = encodings[0]
input_tensor = torch.tensor(encoding, dtype=torch.float32).unsqueeze(0).to(DEVICE)

# Prediction

with torch.no_grad():
    logits = model(input_tensor)
    probs = torch.softmax(logits, dim=1)
    pred_idx = torch.argmax(probs, dim=1).item()
    confidence = probs[0, pred_idx].item()

pred_name = label_encoder.inverse_transform([pred_idx])[0]

#Result
print(f"🧠 Predicted person: {pred_name}")
print(f"🎯 Confidence: {confidence:.4f}")


✅ Model and label encoder loaded successfully. Classes: ['Angelina Jolie' 'Brad Pitt' 'Denzel Washington' 'Hugh Jackman'
 'Jennifer Lawrence' 'Johnny Depp' 'Kate Winslet' 'Leonardo DiCaprio'
 'Megan Fox' 'Natalie Portman' 'Nicole Kidman' 'Robert Downey Jr'
 'Sandra Bullock' 'Scarlett Johansson' 'Tom Cruise' 'Tom Hanks'
 'Will Smith']
🧠 Predicted person: Angelina Jolie
🎯 Confidence: 0.9908


**Batch Prediction: Face Recognition on Multiple Images**

In [ ]:
#Prediction using multiple image

with open(ENCODER_PATH, "rb") as f:
    le = pickle.load(f)

num_classes = len(le.classes_)
model = FaceNetMLP(input_dim=128, num_classes=num_classes)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

for filename in os.listdir(TEST_DIR):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(TEST_DIR, filename)
        img = face_recognition.load_image_file(img_path)
        encs = face_recognition.face_encodings(img)
        if len(encs) == 0:
            print(f"⚠️ No face in {filename}")
            continue
        enc = torch.tensor(encs[0], dtype=torch.float32).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            logits = model(enc)
            probs = torch.softmax(logits, dim=1)
            pred_idx = torch.argmax(probs, dim=1).item()

            conf = probs[0, pred_idx].item()
        name = le.inverse_transform([pred_idx])[0]
        print(f"{filename} → {name} (confidence: {conf:.3f})")


anjel.jpg → Angelina Jolie (confidence: 0.991)
Brad Pitt.jpg → Brad Pitt (confidence: 0.995)
Denzel Washington.jpg → Denzel Washington (confidence: 0.989)
Hugh Jackman.jpg → Hugh Jackman (confidence: 0.996)
Jennifer Lawrence.jpg → Jennifer Lawrence (confidence: 0.947)
Johnny Depp.jpg → Johnny Depp (confidence: 0.992)
Kate Winslet.jpg → Kate Winslet (confidence: 0.954)
Leonardo DiCaprio.jpg → Leonardo DiCaprio (confidence: 0.980)
Megan Fox.jpg → Megan Fox (confidence: 0.933)
Natalie Portman.jpg → Natalie Portman (confidence: 0.997)
Nicole Kidman.jpg → Nicole Kidman (confidence: 0.994)
Robert Downey Jr..jpg → Robert Downey Jr (confidence: 0.997)
Sandra Bullock.jpg → Sandra Bullock (confidence: 0.998)
Scarlett Johansson.jpg → Scarlett Johansson (confidence: 1.000)
Tom Cruise.jpg → Tom Cruise (confidence: 0.997)
Tom Hanks.jpg → Tom Hanks (confidence: 0.986)
Will Smith.jpg → Will Smith (confidence: 0.976)


**Face Recognition with Personal Details Lookup from CSV**

In [ ]:
# Prediction using image and person info

CSV_PATH = "C:/Users/nandhudivya/Desktop/project/actors.csv"

# Model-Loading
with open(ENCODER_PATH, "rb") as f:
    le = pickle.load(f)

num_classes = len(le.classes_)
model = FaceNetMLP(input_dim=128, num_classes=num_classes)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# Loading presonal details-CSV
df = pd.read_csv("C:/Users/nandhudivya/Desktop/project/actors.csv")
df.columns = df.columns.str.strip().str.lower()
df['name'] = df['name'].str.strip().str.lower()
df = df.drop_duplicates(subset='name')
df.set_index('name', inplace=True)


# Test Image
for filename in os.listdir(TEST_DIR):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(TEST_DIR, filename)
        img = face_recognition.load_image_file(img_path)
        encs = face_recognition.face_encodings(img)
        if len(encs) == 0:
            print(f"⚠️ No face in {filename}")
            continue
        enc = torch.tensor(encs[0], dtype=torch.float32).unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            logits = model(enc)
            probs = torch.softmax(logits, dim=1)
            pred_idx = torch.argmax(probs, dim=1).item()
            confidence = probs[0, pred_idx].item()

        pred_name = le.inverse_transform([pred_idx])[0]

        pred_name_clean = pred_name.strip().lower()

        row = df[df.index == pred_name_clean]


        name = le.inverse_transform([pred_idx])[0].strip().lower()
        if name in df.index:
            row = df.loc[name]
            gender = row['gender']
            nationality = row['nationality']
            age = row['age']
        else:
            gender, nationality, age = "Unknown", "Unknown", "Unknown"


        print(f"{filename} → {pred_name} (confidence: {confidence:.3f}) | Age: {age}, gender: {gender}, Nationality: {nationality}")



anjel.jpg → Angelina Jolie (confidence: 0.991) | Age: 53, gender: female, Nationality: American
Brad Pitt.jpg → Brad Pitt (confidence: 0.995) | Age: 45, gender: male, Nationality: American
Denzel Washington.jpg → Denzel Washington (confidence: 0.989) | Age: 38, gender: male, Nationality: American
Hugh Jackman.jpg → Hugh Jackman (confidence: 0.996) | Age: 37, gender: male, Nationality: Australian
Jennifer Lawrence.jpg → Jennifer Lawrence (confidence: 0.947) | Age: 41, gender: female, Nationality: American
Johnny Depp.jpg → Johnny Depp (confidence: 0.992) | Age: 36, gender: male, Nationality: American
Kate Winslet.jpg → Kate Winslet (confidence: 0.954) | Age: 42, gender: female, Nationality: British
Leonardo DiCaprio.jpg → Leonardo DiCaprio (confidence: 0.980) | Age: 34, gender: male, Nationality: American
Megan Fox.jpg → Megan Fox (confidence: 0.933) | Age: 35, gender: female, Nationality: American
Natalie Portman.jpg → Natalie Portman (confidence: 0.997) | Age: 30, gender: female, Nati

**Project Results Summary**

Dataset: 1,702 face encodings for 17 celebrities.

Model: MLP classifier trained on FaceNet embeddings.

Performance: Final accuracy 99.12%; most individuals classified with high precision and recall.

Prediction: Single and batch images predicted correctly with high confidence (93–100%).

Additional Info: Age, gender, and nationality successfully retrieved from CSV for each person.

**Conclusion**

The system accurately recognizes faces and retrieves personal information with high confidence. It demonstrates the effective combination of deep learning embeddings and structured data, providing a reliable and practical solution for face recognition applications.